# Crawl data from [NOAA](https://www.ncei.noaa.gov/support/access-data-service-api-user-documentation)


In [84]:
%pip install requests

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [85]:
import requests

In [86]:
def get_gps_bounding_box(latitude, longitude, deg_lat, deg_lon, verbose):
    """
    Returns a bounding box derived from latitude/longitude deviated by 
    deg_lat and deg_lon as GPS coordinates: n_lat, w_lon, s_lat, e_lon
    All coordinates in decimal degrees. 

    Usage:
        n, w, s, e = get_gps_bounding_box(-88.77425, 37.05635, 1.0, 1.0, False)
        print(n, w, s, e)   # -87.77425 38.05635 -89.77425 36.05635

    """
    # lat1 < -90 or lat1 > 90   0 deg @ equator to +90 at N pole
    # lon1 < -180 or lon1 > 180     0 deg at Greenwich to +180 deg E

    import numpy

    if deg_lat < 0: raise Exception("ERROR: deg_lat passed to get_gps_bounding_box is invalid")
    if not isinstance(deg_lat, float): raise Exception("ERROR: deg_lat passed to get_gps_bounding_box is invalid")
    if deg_lon < 0: raise Exception("ERROR: deg_lon passed to get_gps_bounding_box is invalid")
    if not isinstance(deg_lon, float): raise Exception("ERROR: deg_lon passed to get_gps_bounding_box is invalid")

    if latitude >= 0:
        if latitude + deg_lat >= 90.0:
            n = 90.0 * numpy.sign(latitude)
            s = latitude - deg_lat
        else:   
            n = latitude + deg_lat
            s = latitude - deg_lat

    else:
        # latitude < 0
        if abs(latitude) + deg_lat >= 90.0:
            n = 90.0 * numpy.sign(latitude)
            s = (abs(latitude) - deg_lat) * numpy.sign(latitude)
        else:   
            n = latitude + deg_lat
            s = latitude - deg_lat

    if longitude >= 0:
        if longitude + deg_lon >= 180.0:
            w = 180.0 * numpy.sign(longitude)
            e = longitude - deg_lon
        else:   
            w = longitude + deg_lon
            e = longitude - deg_lon

    else:
        # longitude < 0
        if abs(longitude) + deg_lon >= 180.0:
            w = 180.0 * numpy.sign(longitude)
            e = (abs(longitude) - deg_lon) * numpy.sign(longitude)
        else:   
            w = longitude + deg_lon
            e = longitude - deg_lon

    if verbose: print(round(n,1), round(latitude,1), round(s,1), "\t", round(w,1), round(longitude,1), round(e,1))

    return n, w, s, e

In [87]:
import math

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Returns the distance in meters between the two GPS coordinates in decimal degrees.

    Usage:

        dist_m = haversine_distance(40.440363, -76.126746, 40.440406, -76.121293)
        print("dist_m: ", dist_m)
        # Result is 461.5 m.  The correct result is 461.5 m. 

    """
    # Radius of the Earth in meters
    earth_radius_m = 6371000.0
    
    # Convert latitude and longitude from degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    
    # Differences between the latitudes and longitudes
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    # Haversine formula
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # Calculate the distance
    distance = earth_radius_m * c
    
    return distance

In [88]:

def elevation_by_lat_lon(lat1, lon1):
    """
    Returns the elevation in meters for the specified latitude/longitude, or None
    if the service could not determine the elevation. 

    Uses Google Maps API for elevation, or Open-Elevation if no result.  

    Open-Elevation
    website: https://open-elevation.com/
    docs: https://github.com/Jorl17/open-elevation/blob/master/docs/api.md
    endpoint: curl 'https://api.open-elevation.com/api/v1/lookup?locations=10,10|20,20|41.161758,-8.583933'

    Other API alternatives:
        TessaDEM (https://tessadem.com/) has a free API.  
        GPXZ API is free at 100 requests per day, otherwised $49/mo.  https://www.gpxz.io/#pricing

    Usage:
        elevation_m = elevation_by_lat_lon(38.5288, -78.4383)
        print('Elevation for 38.5288, -78.4383: ', elevation_m, 'meters')

    """

    # api_secrets = {}    

    # if not "api_google_maps" in api_secrets.keys(): raise Exception("ERROR: api_secrets doesn't have the key requested for 'api_google_maps'.")

    # # Try using Google Maps API for elevation
    # url = "https://maps.googleapis.com/maps/api/elevation/json?locations=" + str(lat1) + "," + str(lon1) + "&" + api_secrets['api_google_maps'][0] + "=" + api_secrets['api_google_maps'][1] + ""
    # resp = None
    # #sleep(randint(1,3))     # random pause between 1 and 3 seconds
    # try:
    #     resp = requests.get(url).json() 
    # except Exception as e:
    #     print('ERROR: ' + repr(e), url)

    # if resp is None:
    #     print('maps.googleapis.com failed.  Trying api.open-elevation.com..')
    url = 'https://api.open-elevation.com/api/v1/lookup?locations=' + str(lat1) + ',' + str(lon1)

    print("try " + url);
    # sleep(randint(2,5))     # random pause between 2 and 5 seconds
    try:
        resp = requests.get(url).json() 
    except Exception as e:
        print('ERROR: ' + repr(e), url)

    if not resp:
        return 0

    #print(json.dumps(data, sort_keys=True, indent=2))
    if not 'results' in resp.keys(): 
        print(json.dumps(resp, sort_keys=True, indent=2))
        return None
    results = resp['results'][0]
    if not 'elevation' in results.keys(): return None
    elevation = float(results['elevation'])
    return elevation


In [89]:
import operator

def get_stations_by_bounding_box(lat, lon, verbose):
    """
    Using the latest API endpoint (as of 2023)

    Returns a list of stations along with their GPS coordinates and start/end dates,
    and the computed distance and elevation delta from the target location specified
    by lat,lon that include the dataTypes=TMIN,TMAX,PRCP.  

    Usage:

    stations = get_stations_by_bounding_box(40.44077631612291, -76.12267163754181, False)
    if stations is None:
        print("NO results")
    else:
        print("The best station choice is: ", stations[0][3], "from ", stations[0][6], " to ", stations[0][7])
        # All data in stations:
        print("sort, delta_elevation_m, delta_distance_m, ID, latitude, longitude, dateStart, dateEnd")
        for station in stations:
            print(station)
    
    Output:

    The best station choice is:  USW00013722 from  1944-06-01T00:00:00  to  2023-09-30T23:59:59
    sort, delta_elevation_m, delta_distance_m, ID, latitude, longitude, dateStart, dateEnd
    [8818330.2, 15.8, 556567.6, 'USW00013722', 35.89227, -78.78194, '1944-06-01T00:00:00', '2023-09-30T23:59:59']
    [12044151.2, 83.7, 143881.9, 'USC00286055', 40.47282, -74.42259, '1968-06-01T00:00:00', '2023-08-31T23:59:59']
    [31981786.2, 113.8, 281154.6, 'USC00303184', 42.8766, -77.0307, '1969-01-01T00:00:00', '2023-08-31T23:59:59']
    [60606995.9, 67.5, 897346.3, 'USC00177238', 45.08533, -67.1205, '1994-11-01T00:00:00', '2023-08-31T23:59:59']
    [100554773.3, 107.7, 933870.5, 'USC00122309', 38.4558, -86.6983, '1955-01-01T00:00:00', '2023-07-31T23:59:59']
    [110337972.5, 120.7, 913908.0, 'USC00129222', 41.4439, -86.9294, '1961-01-01T00:00:00', '2023-08-31T23:59:59']
    [280852969.2, 189.9, 1479068.0, 'USC00218450', 44.9902, -93.17995, '1960-10-01T00:00:00', '2023-08-31T23:59:59']
    [430710417.9, 251.6, 1711708.5, 'USC00255362', 41.143, -96.4808, '1968-11-01T00:00:00', '2023-08-31T23:59:59']
    [1233513339.3, 579.1, 2130165.0, 'USC00395544', 44.5171, -101.6184, '1911-10-01T00:00:00', '2023-08-31T23:59:59']
    [3908089408.1, 1356.5, 2880981.5, 'USC00241047', 45.6729, -111.1547, '1966-11-01T00:00:00', '2023-08-31T23:59:59']
    
    """

    gps_n, gps_w, gps_s, gps_e = get_gps_bounding_box(lat, lon, 1.0, 1.0, verbose)

    #url = "https://www.ncei.noaa.gov/access/services/search/v1/data?dataset=global-summary-of-the-month&boundingBox=35.462327,-82.563951,35.412327,-82.513951&dataTypes=TMIN,TMAX,PRCP&limit=10&offset=0"
    url = "https://www.ncei.noaa.gov/access/services/search/v1/data?dataset=global-summary-of-the-month"
    url += "&bbox=" + str(gps_n) + "," + str(gps_w) + "," + str(gps_s) + "," + str(gps_e)
    url += "&dataTypes=TMIN,TMAX,TAVG,PRCP"
    url += "&limit=10&offset=0"
    if verbose: print(url)

    headers = {'token': 'your_NOAA_token'}
    
    #sleep(randint(1,3))     # random pause between 1 and 3 seconds
    try:
        req = requests.get(url, data=None, json=None, headers=None)
    except Exception as e:
        print('ERROR: ' + repr(e), ' fn()', url)
        return None
    
    if not req.status_code == 200: 
        if req.status_code == 429:
            raise Exception('ERROR: HTTP 429 Too Many Requests! rate limiting.  resp.status_code = ', str(req.status_code), req.text)
        else:
            print('\tERROR: resp.status_code = ', str(req.status_code), req.text)
            return None

    resp = req.json()
    if len(resp) == 0:
        print("\tNo results from query ", url)
        return None
    
    #print(json.dumps(resp, sort_keys=False, indent=2))
    station_data = []
    #if 'data' in resp.keys():
    if not 'results' in resp.keys(): raise Exception("ERROR: key 'results' not in response")
    results = resp['results']
    for result in results:
        if not 'endDate' in result.keys(): raise Exception("ERROR: key 'endDate' not in response (result)")
        if not 'startDate' in result.keys(): raise Exception("ERROR: key 'startDate' not in response (result)")
        if not 'centroid' in result.keys(): raise Exception("ERROR: key 'centroid' not in response (result)")
        if not 'point' in result['centroid'].keys(): raise Exception("ERROR: key 'point' not in response (result['centroid'])")
        centroid = result['centroid']
        longitude = float(centroid['point'][0])
        latitude = float(centroid['point'][1])
        if not 'name' in result.keys(): raise Exception("ERROR: key 'name' not in response (result)")
        if not 'location' in result.keys(): raise Exception("ERROR: key 'location' not in response (result)")
        if not 'id' in result.keys(): raise Exception("ERROR: key 'id' not in response (result)")
        if not 'dataTypesCount' in result.keys(): raise Exception("ERROR: key 'dataTypesCount' not in response (result)")       
        if not 'boundingPoints' in result.keys(): raise Exception("ERROR: key 'boundingPoints' not in response (result)")        
        if not 'stations' in result.keys(): raise Exception("ERROR: key 'stations' not in response (result)")
        stations = result['stations']
        for station in stations:
            if not 'name' in station.keys(): raise Exception("ERROR: key 'name' not in response (stations)")
            if not 'id' in station.keys(): raise Exception("ERROR: key 'id' not in response (stations)")
            delta_distance_m = haversine_distance(lat, lon, latitude, longitude)
            delta_elevation_m = abs(elevation_by_lat_lon(lat, lon) - elevation_by_lat_lon(latitude, longitude))
            sort = delta_distance_m * delta_elevation_m
            if verbose: print(station['id'], "\t", latitude, longitude, "\t", round(delta_distance_m,1), round(delta_elevation_m,1), result['startDate'], "\t", result['endDate'])
            station_data.append([round(sort,1), round(delta_elevation_m,1), round(delta_distance_m,1), station['id'], latitude, longitude, result['startDate'], result['endDate']])
    
    # Sort the data by delta_distance_m * delta_elevation
    station_data.sort(reverse=False, key=operator.itemgetter(0))
    return station_data

In [ ]:
gps_n, gps_w, gps_s, gps_e = 20.4, 103.2, 23.0, 107.4
)
#url = "https://www.ncei.noaa.gov/access/services/search/v1/data?dataset=global-summary-of-the-month&boundingBox=35.462327,-82.563951,35.412327,-82.513951&dataTypes=TMIN,TMAX,PRCP&limit=10&offset=0"
url = "https://www.ncei.noaa.gov/access/services/search/v1/data?dataset=global-summary-of-the-month"
url += "&bbox=" + str(gps_n) + "," + str(gps_w) + "," + str(gps_s) + "," + str(gps_e)
url += "&dataTypes=TMIN,TMAX,TAVG,PRCP"
url += "&limit=10&offset=0"


url

90.0 105 104.0 	 21.0 20 19.0


'https://www.ncei.noaa.gov/access/services/search/v1/data?dataset=global-summary-of-the-month&bbox=90.0,21.0,104.0,19.0&dataTypes=TMIN,TMAX,TAVG,PRCP&limit=10&offset=0'

In [ ]:
stations = get_stations_by_bounding_box(20.9642809, 105.7935119, False)
# stations = get_stations_by_bounding_box(40.44077631612291, -76.12267163754181, False)

	ERROR: resp.status_code =  500 {"errorCode":500,"errorMessage":"Internal Server Error","errors":[{"field":null,"message":"An unexpected error occurred while servicing your request.","value":null}]}


In [94]:
stations

[[9768369.8,
  1.0,
  9768369.8,
  'USC00514500',
  21.6793,
  -157.9453,
  '1980-03-01T00:00:00',
  '2024-09-30T23:59:59'],
 [9772167.8,
  1.0,
  9772167.8,
  'USW00022551',
  21.30555,
  -158.06582,
  '1999-09-01T00:00:00',
  '2024-10-31T23:59:59'],
 [19511712.1,
  2.0,
  9755856.0,
  'USC00519195',
  21.5747,
  -158.1204,
  '1908-01-01T00:00:00',
  '2001-11-30T23:59:59'],
 [29350700.3,
  3.0,
  9783566.8,
  'USW00022521',
  21.32402,
  -157.93946,
  '1940-06-01T00:00:00',
  '2024-10-31T23:59:59'],
 [29384524.6,
  3.0,
  9794841.5,
  'USW00022519',
  21.45045,
  -157.76794,
  '1942-01-01T00:00:00',
  '2024-10-31T23:59:59'],
 [59678637.7,
  6.0,
  9946439.6,
  'USW00022516',
  20.88871,
  -156.43453,
  '1905-01-01T00:00:00',
  '2024-11-30T23:59:59'],
 [1224090238.6,
  124.0,
  9871695.5,
  'USW00022534',
  21.15448,
  -157.09612,
  '1949-10-01T00:00:00',
  '2024-10-31T23:59:59'],
 [1479251285.5,
  151.0,
  9796366.1,
  'USC00516128',
  21.3331,
  -157.8025,
  '1975-03-01T00:00:00',
  